# K₇ Spectral v4: Δ₀ vs Δ₁ Comparison

**Building on v3**: Now that we have a validated methodology (3 grids converge perfectly),
let's compare the **scalar Laplacian** (Δ₀) vs **Hodge Laplacian on 1-forms** (Δ₁).

## Theory Reminder

- **Δ₀** acts on functions (0-forms): Δ₀f = -div(grad f)
- **Δ₁** acts on 1-forms: Δ₁ω = (dδ + δd)ω

For Yang-Mills, the relevant operator is **Δ₁** (gauge fields are 1-forms)!

---

In [ ]:
# @title 1. Setup
import numpy as np
from scipy.sparse.linalg import LinearOperator, eigsh
from scipy.sparse import kron, eye, diags
from scipy import sparse
import time
import json

# GIFT Constants
DIM = 7
H_STAR = 99
DIM_G2 = 14
DET_G = 65/32
G_II = DET_G ** (1/7)  # ≈ 1.1065

print("="*60)
print("K₇ SPECTRAL v4: Δ₀ vs Δ₁ COMPARISON")
print("="*60)
print(f"Metric: g_ii = {G_II:.6f}, det(g) = {DET_G}")

In [ ]:
# @title 2. 1D Operators (Building Blocks)

def make_D1_periodic(N: int, h: float) -> sparse.csr_matrix:
    """
    First derivative operator (periodic BC).
    Central difference: (f[i+1] - f[i-1]) / (2h)
    """
    diag_p1 = np.ones(N) / (2*h)
    diag_m1 = -np.ones(N) / (2*h)
    D = diags([diag_m1, diag_p1], [-1, 1], shape=(N, N), format='csr')
    # Periodic: wrap around
    D = D.tolil()
    D[0, N-1] = -1 / (2*h)
    D[N-1, 0] = 1 / (2*h)
    return D.tocsr()

def make_D2_periodic(N: int, h: float) -> sparse.csr_matrix:
    """
    Second derivative operator (periodic BC).
    (f[i+1] - 2f[i] + f[i-1]) / h²
    """
    diag_0 = -2 * np.ones(N) / (h*h)
    diag_pm1 = np.ones(N) / (h*h)
    D2 = diags([diag_pm1, diag_0, diag_pm1], [-1, 0, 1], shape=(N, N), format='csr')
    # Periodic
    D2 = D2.tolil()
    D2[0, N-1] = 1 / (h*h)
    D2[N-1, 0] = 1 / (h*h)
    return D2.tocsr()

print("1D operators defined.")

In [ ]:
# @title 3. Scalar Laplacian Δ₀ (7D)

def build_laplacian_0_sparse(N: int, L: float = 2*np.pi, g_diag: np.ndarray = None) -> sparse.csr_matrix:
    """
    Build sparse scalar Laplacian Δ₀ on 7D torus.
    
    Δ₀ = Σᵢ (1/gᵢᵢ) ∂²/∂xᵢ²
    
    For constant diagonal metric.
    """
    if g_diag is None:
        g_diag = np.ones(DIM)
    
    h = L / N
    D2 = make_D2_periodic(N, h)
    I = eye(N, format='csr')
    
    # Build 7D Laplacian via Kronecker products
    # Δ = Σᵢ (1/gᵢᵢ) (I ⊗ ... ⊗ D² ⊗ ... ⊗ I)
    
    print(f"Building Δ₀ for N={N}^7 = {N**7:,} points...")
    
    L0 = sparse.csr_matrix((N**DIM, N**DIM), dtype=np.float64)
    
    for d in range(DIM):
        # Build I ⊗ ... ⊗ D² ⊗ ... ⊗ I
        ops = [I] * DIM
        ops[d] = D2
        
        term = ops[0]
        for op in ops[1:]:
            term = kron(term, op, format='csr')
        
        # Weight by inverse metric
        L0 = L0 + (1.0 / g_diag[d]) * term
        
        if d == 0:
            print(f"  Dimension {d}: term shape = {term.shape}, nnz = {term.nnz:,}")
    
    print(f"  Total: shape = {L0.shape}, nnz = {L0.nnz:,}")
    return -L0  # Return -Δ (positive semi-definite)

print("Δ₀ builder ready.")

In [ ]:
# @title 4. Hodge Laplacian Δ₁ on 1-Forms (7D)

def build_laplacian_1_sparse(N: int, L: float = 2*np.pi, g_diag: np.ndarray = None) -> sparse.csr_matrix:
    """
    Build sparse Hodge Laplacian Δ₁ on 1-forms for 7D torus.
    
    For a diagonal metric, the Hodge Laplacian on 1-forms is:
    
    (Δ₁ω)ᵢ = Σⱼ (1/gⱼⱼ) ∂²ωᵢ/∂xⱼ² - (1/gᵢᵢ) ∂/∂xᵢ(Σⱼ (1/gⱼⱼ) ∂ωⱼ/∂xⱼ) + Ricci terms
    
    For CONSTANT diagonal metric, this simplifies significantly:
    - No Christoffel symbols (constant metric)
    - No Ricci terms (flat)
    
    Δ₁ = dδ + δd where:
    - d: exterior derivative
    - δ = *d*: codifferential
    
    On flat torus with constant metric g_ii = c:
    (Δ₁ω)ᵢ = (1/c) Σⱼ ∂²ωᵢ/∂xⱼ²  (same as scalar!)
    
    But let's build it properly from d and δ.
    """
    if g_diag is None:
        g_diag = np.ones(DIM)
    
    h = L / N
    D1 = make_D1_periodic(N, h)
    D2 = make_D2_periodic(N, h)
    I = eye(N, format='csr')
    
    n_scalar = N ** DIM
    n_1form = DIM * n_scalar  # 7 components per point
    
    print(f"Building Δ₁ for N={N}^7 = {n_scalar:,} points, {n_1form:,} 1-form components...")
    
    # For constant diagonal metric on torus:
    # Δ₁ decomposes into blocks
    # Each component ωᵢ satisfies: (Δ₁ω)ᵢ = (1/c) Δ_flat ωᵢ + coupling terms
    
    # Actually, for a FLAT torus with constant metric:
    # Δ₁ = Δ₀ ⊗ I₇ (each component evolves independently)
    # This is because there's no curvature coupling!
    
    # Build scalar Laplacian first
    L0_flat = sparse.csr_matrix((n_scalar, n_scalar), dtype=np.float64)
    
    for d in range(DIM):
        ops = [I] * DIM
        ops[d] = D2
        term = ops[0]
        for op in ops[1:]:
            term = kron(term, op, format='csr')
        L0_flat = L0_flat + (1.0 / g_diag[d]) * term
    
    # For constant metric, Δ₁ = Δ₀ ⊗ I_7 on the 7 components
    # BUT with different metric weights for each component direction
    
    # More precisely, for Hodge Laplacian on 1-forms with diagonal metric:
    # The i-th component of a 1-form has a factor √(g_ii) in the Hodge star
    # This leads to: (Δ₁)_{ii block} = (1/g_ii) * Δ_flat (component-wise)
    
    # Build block-diagonal Δ₁
    blocks = []
    for i in range(DIM):
        # For component i, the effective Laplacian has an extra 1/g_ii factor
        # from the Hodge star metric weight
        block = -L0_flat  # This is already (1/g_jj) weighted
        blocks.append(block)
    
    L1 = sparse.block_diag(blocks, format='csr')
    
    print(f"  Total: shape = {L1.shape}, nnz = {L1.nnz:,}")
    
    return L1

print("Δ₁ builder ready.")

In [ ]:
# @title 5. Proper Hodge Laplacian via d and δ

def build_hodge_laplacian_1_proper(N: int, L: float = 2*np.pi, g_diag: np.ndarray = None) -> sparse.csr_matrix:
    """
    Build Hodge Laplacian Δ₁ = dδ + δd properly.
    
    For 1-forms on 7D torus:
    - d: Ω¹ → Ω²  (exterior derivative)
    - δ: Ω¹ → Ω⁰  (codifferential = -div for 1-forms)
    
    δω = -Σᵢ (1/√g) ∂/∂xᵢ(√g gⁱⁱ ωᵢ) = -Σᵢ (1/gᵢᵢ) ∂ωᵢ/∂xᵢ  (for const metric)
    
    (dδω)ᵢ = ∂/∂xᵢ(δω) = -∂/∂xᵢ Σⱼ (1/gⱼⱼ) ∂ωⱼ/∂xⱼ
    
    For δd, we need d first:
    (dω)ᵢⱼ = ∂ωⱼ/∂xᵢ - ∂ωᵢ/∂xⱼ
    
    Then δ on 2-forms... this gets complicated.
    
    For FLAT torus with CONSTANT metric, use Weitzenböck:
    Δ₁ = ∇*∇ + Ric
    
    With Ric = 0 (flat), we get Δ₁ = ∇*∇ (rough Laplacian)
    which equals the component-wise scalar Laplacian!
    """
    if g_diag is None:
        g_diag = np.ones(DIM)
    
    h = L / N
    D1 = make_D1_periodic(N, h)
    D2 = make_D2_periodic(N, h)
    I_1d = eye(N, format='csr')
    
    n_scalar = N ** DIM
    n_1form = DIM * n_scalar
    
    print(f"Building proper Hodge Δ₁ for N={N}^7...")
    
    # Build ∂/∂xᵢ operators on the scalar space
    D_ops = []  # D_ops[i] = ∂/∂xᵢ
    D2_ops = []  # D2_ops[i] = ∂²/∂xᵢ²
    
    for d in range(DIM):
        ops = [I_1d] * DIM
        ops[d] = D1
        D_d = ops[0]
        for op in ops[1:]:
            D_d = kron(D_d, op, format='csr')
        D_ops.append(D_d)
        
        ops[d] = D2
        D2_d = ops[0]
        for op in ops[1:]:
            D2_d = kron(D2_d, op, format='csr')
        D2_ops.append(D2_d)
    
    # Identity on scalar space
    I_scalar = eye(n_scalar, format='csr')
    
    # Build Δ₁ as a 7×7 block matrix (each block is n_scalar × n_scalar)
    # (Δ₁)ᵢⱼ block operates: (Δ₁ω)ᵢ contribution from ωⱼ
    
    # From Weitzenböck on flat space:
    # (Δ₁ω)ᵢ = Σⱼ (1/gⱼⱼ) ∂²ωᵢ/∂xⱼ²  (diagonal blocks, i=j)
    # Plus cross terms from dδ:
    # (dδω)ᵢ = -Σⱼ (1/gⱼⱼ) ∂²ωⱼ/∂xᵢ∂xⱼ
    
    # So the full operator is:
    # (Δ₁ω)ᵢ = Σⱼ (1/gⱼⱼ) [∂²ωᵢ/∂xⱼ² - ∂²ωⱼ/∂xᵢ∂xⱼ] + (1/gᵢᵢ) ∂²ωᵢ/∂xᵢ²
    #        = Σⱼ (1/gⱼⱼ) ∂²ωᵢ/∂xⱼ² - Σⱼ (1/gⱼⱼ) ∂²ωⱼ/∂xᵢ∂xⱼ
    
    # Build block matrix
    blocks = [[None for _ in range(DIM)] for _ in range(DIM)]
    
    for i in range(DIM):
        for j in range(DIM):
            if i == j:
                # Diagonal: Σₖ (1/gₖₖ) ∂²/∂xₖ²
                block = sparse.csr_matrix((n_scalar, n_scalar), dtype=np.float64)
                for k in range(DIM):
                    block = block + (1.0 / g_diag[k]) * D2_ops[k]
                blocks[i][j] = -block
            else:
                # Off-diagonal: -(1/gⱼⱼ) ∂²/∂xᵢ∂xⱼ from dδ term
                # ∂²/∂xᵢ∂xⱼ = D_ops[i] @ D_ops[j]
                cross = D_ops[i] @ D_ops[j]
                blocks[i][j] = (1.0 / g_diag[j]) * cross
    
    # Assemble block matrix
    L1 = sparse.bmat(blocks, format='csr')
    
    print(f"  Shape: {L1.shape}, nnz: {L1.nnz:,}")
    
    return L1

print("Proper Hodge Δ₁ builder ready.")

In [ ]:
# @title 6. Test on Small Grid First

N_TEST = 5
L = 2 * np.pi

print("="*60)
print(f"TEST RUN: N = {N_TEST}")
print("="*60)

# Flat metric first (calibration)
g_flat = np.ones(DIM)
g_G2 = np.full(DIM, G_II)

print("\n--- Δ₀ (Scalar) on Flat T⁷ ---")
t0 = time.time()
L0_flat = build_laplacian_0_sparse(N_TEST, L, g_flat)
eigs_0_flat = eigsh(L0_flat, k=15, which='SM', return_eigenvectors=False)
eigs_0_flat = np.sort(eigs_0_flat)
print(f"  Time: {time.time()-t0:.2f}s")
print(f"  λ₀ = {eigs_0_flat[0]:.6f} (should be ~0)")
print(f"  λ₁ = {eigs_0_flat[1]:.6f} (should be ~1)")

print("\n--- Δ₁ (1-Forms) on Flat T⁷ ---")
t0 = time.time()
L1_flat = build_hodge_laplacian_1_proper(N_TEST, L, g_flat)
eigs_1_flat = eigsh(L1_flat, k=20, which='SM', return_eigenvectors=False)
eigs_1_flat = np.sort(eigs_1_flat)
print(f"  Time: {time.time()-t0:.2f}s")
print(f"  First 10 eigenvalues: {eigs_1_flat[:10]}")
# For flat T⁷, Δ₁ should have 7 zero modes (harmonic 1-forms for b₁=7)
nonzero_1 = eigs_1_flat[eigs_1_flat > 1e-6]
print(f"  First nonzero λ₁ = {nonzero_1[0] if len(nonzero_1)>0 else 'N/A'}")

In [ ]:
# @title 7. Compare Δ₀ vs Δ₁ with G₂ Metric

print("\n" + "="*60)
print("COMPARISON: Δ₀ vs Δ₁ WITH G₂ METRIC")
print("="*60)
print(f"\nMetric: g_ii = {G_II:.6f}")

print("\n--- Δ₀ (Scalar) with G₂ ---")
t0 = time.time()
L0_G2 = build_laplacian_0_sparse(N_TEST, L, g_G2)
eigs_0_G2 = eigsh(L0_G2, k=15, which='SM', return_eigenvectors=False)
eigs_0_G2 = np.sort(eigs_0_G2)
print(f"  Time: {time.time()-t0:.2f}s")
print(f"  λ₁(Δ₀) = {eigs_0_G2[1]:.6f}")

print("\n--- Δ₁ (1-Forms) with G₂ ---")
t0 = time.time()
L1_G2 = build_hodge_laplacian_1_proper(N_TEST, L, g_G2)
eigs_1_G2 = eigsh(L1_G2, k=30, which='SM', return_eigenvectors=False)
eigs_1_G2 = np.sort(eigs_1_G2)
print(f"  Time: {time.time()-t0:.2f}s")
nonzero_1_G2 = eigs_1_G2[eigs_1_G2 > 1e-6]
lambda1_Delta1 = nonzero_1_G2[0] if len(nonzero_1_G2) > 0 else eigs_1_G2[7]
print(f"  λ₁(Δ₁) = {lambda1_Delta1:.6f}")

# Calibration
lambda1_0_flat = eigs_0_flat[1]
lambda1_1_flat = nonzero_1[0] if len(nonzero_1) > 0 else 1.0

scale_0 = 1.0 / lambda1_0_flat
scale_1 = 1.0 / lambda1_1_flat

lambda1_0_calibrated = eigs_0_G2[1] * scale_0
lambda1_1_calibrated = lambda1_Delta1 * scale_1

print(f"\n--- Calibrated Results ---")
print(f"  λ₁(Δ₀) calibrated = {lambda1_0_calibrated:.6f}")
print(f"  λ₁(Δ₁) calibrated = {lambda1_1_calibrated:.6f}")

print(f"\n--- Products with H* ---")
product_0 = lambda1_0_calibrated * H_STAR
product_1 = lambda1_1_calibrated * H_STAR
print(f"  λ₁(Δ₀) × H* = {product_0:.4f}")
print(f"  λ₁(Δ₁) × H* = {product_1:.4f}")
print(f"  Target:       14")
print(f"\n  Ratio Δ₁/Δ₀ = {product_1/product_0:.4f}")

In [ ]:
# @title 8. Larger Grid (if time permits)

RUN_LARGE = True  # Set to True for better accuracy

if RUN_LARGE:
    N_LARGE = 7
    print(f"\n" + "="*60)
    print(f"LARGER GRID: N = {N_LARGE}")
    print("="*60)
    
    # Flat calibration
    print("\n--- Flat T⁷ Calibration ---")
    L0_flat_L = build_laplacian_0_sparse(N_LARGE, L, g_flat)
    eigs_0_flat_L = eigsh(L0_flat_L, k=10, which='SM', return_eigenvectors=False)
    scale_0_L = 1.0 / np.sort(eigs_0_flat_L)[1]
    print(f"  Scale factor: {scale_0_L:.6f}")
    
    # Δ₀ with G₂
    print("\n--- Δ₀ with G₂ ---")
    L0_G2_L = build_laplacian_0_sparse(N_LARGE, L, g_G2)
    eigs_0_G2_L = eigsh(L0_G2_L, k=10, which='SM', return_eigenvectors=False)
    lambda1_0_L = np.sort(eigs_0_G2_L)[1] * scale_0_L
    print(f"  λ₁(Δ₀) = {lambda1_0_L:.6f}")
    print(f"  λ₁(Δ₀) × H* = {lambda1_0_L * H_STAR:.4f}")
    
    # Δ₁ with G₂
    print("\n--- Δ₁ with G₂ ---")
    L1_G2_L = build_hodge_laplacian_1_proper(N_LARGE, L, g_G2)
    eigs_1_G2_L = eigsh(L1_G2_L, k=30, which='SM', return_eigenvectors=False)
    eigs_1_G2_L = np.sort(eigs_1_G2_L)
    nonzero_L = eigs_1_G2_L[eigs_1_G2_L > 1e-6]
    lambda1_1_L_raw = nonzero_L[0] if len(nonzero_L) > 0 else eigs_1_G2_L[7]
    lambda1_1_L = lambda1_1_L_raw * scale_0_L  # Same scale
    print(f"  λ₁(Δ₁) = {lambda1_1_L:.6f}")
    print(f"  λ₁(Δ₁) × H* = {lambda1_1_L * H_STAR:.4f}")
else:
    print("Large grid skipped. Set RUN_LARGE = True to run.")

In [ ]:
# @title 9. Analysis

print("\n" + "="*60)
print("ANALYSIS")
print("="*60)

print("""
For CONSTANT diagonal metric g_ii = c on a FLAT torus:

1. Δ₀ (scalar): λ₁ = 1/c (as we verified in v3)

2. Δ₁ (1-forms) with Weitzenböck identity:
   Δ₁ = ∇*∇ + Ric
   
   For flat space, Ric = 0, so Δ₁ = ∇*∇
   The connection Laplacian ∇*∇ on 1-forms equals the component-wise
   scalar Laplacian for flat metric!
   
   So: λ₁(Δ₁) = λ₁(Δ₀) = 1/c for constant metric.

BUT: The cross terms from dδ can modify this!
Let's check what the numerical results say...
""")

if RUN_LARGE:
    ratio = lambda1_1_L / lambda1_0_L
    print(f"Numerical ratio λ₁(Δ₁)/λ₁(Δ₀) = {ratio:.6f}")
    
    if abs(ratio - 1.0) < 0.01:
        print("\n→ Δ₁ ≈ Δ₀ (as expected for flat constant metric)")
        print("→ The difference must come from TOPOLOGY, not the operator!")
    else:
        print(f"\n→ Δ₁ ≠ Δ₀! Ratio = {ratio:.4f}")
        print("→ The cross terms in Hodge Laplacian matter!")

In [ ]:
# @title 10. Export Results

results = {
    'version': 'v4_delta0_vs_delta1',
    'grid_size': N_LARGE if RUN_LARGE else N_TEST,
    'metric': {
        'g_ii': G_II,
        'det_g': DET_G,
    },
    'delta_0': {
        'lambda1_calibrated': float(lambda1_0_L if RUN_LARGE else lambda1_0_calibrated),
        'product': float((lambda1_0_L if RUN_LARGE else lambda1_0_calibrated) * H_STAR),
    },
    'delta_1': {
        'lambda1_calibrated': float(lambda1_1_L if RUN_LARGE else lambda1_1_calibrated),
        'product': float((lambda1_1_L if RUN_LARGE else lambda1_1_calibrated) * H_STAR),
    },
    'ratio_delta1_delta0': float((lambda1_1_L/lambda1_0_L) if RUN_LARGE else (lambda1_1_calibrated/lambda1_0_calibrated)),
    'H_star': H_STAR,
    'target': 14,
}

with open('K7_spectral_v4_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to K7_spectral_v4_results.json")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\n  λ₁(Δ₀) × H* = {results['delta_0']['product']:.4f}")
print(f"  λ₁(Δ₁) × H* = {results['delta_1']['product']:.4f}")
print(f"  Ratio:        {results['ratio_delta1_delta0']:.4f}")
print(f"  Target:       14")
print("\n" + "="*60)